In [22]:
import os
import shutil
from time import sleep
import time
from linecache import getline
from IPython.display import clear_output



class File:
    
    def __init__(self, file_name):
        self.delete_me = False
        self.file_name = file_name
        file_name, suffix = file_name.split(".")
        if(suffix != "out"): self.delete_me = True ; return
        self.slurm_name = file_name.split("_")[0]
            
        self.name_for_user = "Starting..."
        lines = open(self.file_name).readlines()
        for i, line in enumerate(lines):
            if(line[:5] == "name:" and lines[i+1][0] != "\t"):
                self.name_for_user = lines[i+1][:-1]
                break
        
        self.last_line = "Starting..."
        if(len(lines) != 0):
            last = -1
            while(lines[last] == "\n"): last -= 1
            self.last_line = lines[last]
            
        self.last_trim_time = time.time()
                        
    def trim_file(self):
        """Keeps only the first 1000 and last 1000 lines of the file."""
        try:
            with open(self.file_name, 'r') as f:
                lines = f.readlines()
            if len(lines) > 2000:  # Only trim if the file is too large
                with open(self.file_name, 'w') as f:
                    f.writelines(lines[:2000])  # Keep first 1000 lines
                    f.write("\n... (middle trimmed) ...\n")  # Indicate trimming
                    f.writelines(lines[-2000:])  # Keep last 1000 lines
        except Exception as e:
            print(f"Error trimming {self.file_name}: {e}")

    def check_trim(self):
        """Checks if an hour has passed and trims the file if needed."""
        if time.time() - self.last_trim_time > 3600:  # 3600 seconds = 1 hour
            self.trim_file()
            self.last_trim_time = time.time()
            
    def done(self):
        if(self.last_line.endswith("Done!\n")): return(True)
        return(False)
        
    def __lt__(self, other): return(self.slurm_name < other.slurm_name)
            
    

first = True
while(True):
    file_names = [f for f in os.listdir() if f[:5]=="slurm"]
    
    files = []
    for name in file_names:
        file = File(name)
        if(file.delete_me): pass 
        else:               files.append(file)
    files.sort()

    ongoing = []              ; finished = []
    
    for file in files:
        done = file.done() 
        #file.check_trim()
        if(done): finished.append(file)
        else:     ongoing.append(file)
    
    if(not first):
        print("\n\n")
        for i in range(10):
            print(i+1, end = "... ")
            sleep(1)
    first = False
    
    clear_output(wait=True)
    
    uniform_length = 0
    for file in ongoing + finished:
        if(file.last_line != "\n"):
            label = "{} ({}):".format(file.name_for_user, file.file_name)
            uniform_length = max([uniform_length, len(label)])
                
    print("Ongoing:")
    if(len(ongoing) == 0): print("None.")
    for file in ongoing: 
        label = "{} ({}):".format(file.name_for_user, file.file_name)
        separation = " " * (uniform_length - len(label))
        last_line = file.last_line if file.last_line[-1] != "\n" else file.last_line[:-1]
        print("{}{}\t{}".format(label, separation, last_line))

    print("\n\n\nFinished:")
    if(len(finished) == 0): 
        print("None.")
    else:
        for i, file in enumerate(finished):
            label = "{} ({}):".format(file.name_for_user, file.file_name) 
            separation = " " * (uniform_length - len(label))
            last_line = file.last_line if file.last_line[-1] != "\n" else file.last_line[:-1]
            print("{}{}\t{}".format(label, separation, last_line))
            
    finished_names = [file.name_for_user for file in finished]
    ongoing_names = [file.name_for_user for file in ongoing]

Ongoing:
e (slurm-61766966.out):      	1 day, 8:06:15.329885 (2:38:17.670115 left):	 92  92  94  94  95  95  95  95  96  96  96  97  97  97  97  97  97  98  98  98  98  99  99  99  99  99  99  99  ## 100 100.
eqt_1 (slurm-61766969.out):  	1 day, 8:06:09.142504 (3:05:00.857496 left):	 91  94  95  98  99  ## 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100.
eqt_2 (slurm-61766970.out):  	1 day, 8:06:15.170130 (4:16:27.829870 left):	 88  91  91  92  92  93  93  93  93  93  93  93  93  94  94  94  94  94  95  95  95  95  96  97  97  97  ## 100 100 100 100.
eqt_4 (slurm-61766972.out):  	1 day, 8:06:17.750379 (2:32:00.249621 left):	 92  95  96  96  96  97  97  97  97  98  98  98  98  98  99  99  99  99  99  99  ## 100 100 100 100 100 100 100 100 100 100.
eqt_5 (slurm-61766973.out):  	1 day, 8:05:44.951664 (2:27:34.048336 left):	 92  95  95  96  97  97  97  98  98  98  98  98  98  98  98  98  98  98  99  99  99  99  99  ## 100 100 100 100 100 

KeyboardInterrupt: 

In [24]:
#arg_title_func = "finish_dicts"
arg_title_func = "plotting"
#arg_title_func = "plotting_composition"

if len(finished_names) > 0:
    print("\n\nFINISHED Generated arg_title:")
    print(f"singularity exec maze.sif python communication/{arg_title_func}.py --comp deigo --arg_title ___{'+'.join(finished_names)}___ --arg_name {arg_title_func}")
    print()
    for finished_name in finished_names:
        print(f"singularity exec maze.sif python communication/{arg_title_func}.py --comp deigo --arg_title ___{finished_name}___ --arg_name {arg_title_func}")
    print("\n\nFINISHED Generated arg_list:")
    print(f"bash communication/bash/maze.sh 'arg_list=({' '.join(finished_names)}' 'comp=deigo' 'agents=0'")
    print()
    for finished_name in finished_names:
        print(f"bash communication/bash/maze.sh 'arg_list=({finished_name}' 'comp=deigo' 'agents=0'")
        
        

print("\n\nONGOING Generated arg_title:")
print(f"singularity exec maze.sif python communication/{arg_title_func}.py --comp deigo --arg_title ___{'+'.join(ongoing_names)}___ --arg_name {arg_title_func} --temp True")
print()
for ongoing_name in ongoing_names:
    print(f"singularity exec maze.sif python communication/{arg_title_func}.py --comp deigo --arg_title ___{ongoing_name}___ --arg_name {arg_title_func} --temp True")
print("\n\nONGOING Generated arg_list:")
print(f"bash communication/bash/maze.sh 'arg_list=({' '.join(ongoing_names)}' 'comp=deigo' 'agents=0'")
print()
for ongoing_name in ongoing_names:
    print(f"bash communication/bash/maze.sh 'arg_list=({ongoing_name}' 'comp=deigo' 'agents=0'")
    
    
    
print("\n\nALL Generated arg_title:")
print(f"singularity exec maze.sif python communication/{arg_title_func}.py --comp deigo --arg_title ___{'+'.join(finished_names) + '+' + '+'.join(ongoing_names)}___ --arg_name {arg_title_func} --temp True")
print()
for name in finished_names + ongoing_names:
    print(f"singularity exec maze.sif python communication/{arg_title_func}.py --comp deigo --arg_title ___{name}___ --arg_name {arg_title_func} --temp True")
print("\n\nALL Generated arg_list:")
print(f"bash communication/bash/maze.sh 'arg_list=({' '.join(finished_names) + ' ' + ' '.join(ongoing_names)}' 'comp=deigo' 'agents=0'")
print()
for ongoing_name in finished_names + ongoing_names:
    print(f"bash communication/bash/maze.sh 'arg_list=({ongoing_name}' 'comp=deigo' 'agents=0'")




FINISHED Generated arg_title:
singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___ec+ef+eqt_3+eqt_6+eqt_7+eqt_8+eqt_10+eqt_11+eqt_12+ecqt_1+ecqt_3+ecqt_4+ecqt_5+ecqt_8+ecqt_9+ecqt_10+efqt_3+efqt_5+efqt_12___ --arg_name plotting

singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___ec___ --arg_name plotting
singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___ef___ --arg_name plotting
singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___eqt_3___ --arg_name plotting
singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___eqt_6___ --arg_name plotting
singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___eqt_7___ --arg_name plotting
singularity exec maze.sif python communication/plotting.py --comp deigo --arg_title ___eqt_8___ --arg_name plotting
singularity exec maze.sif python communicati